Project intro: help an e-commerce website to decide implement new page or not

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
df=pd.read_csv("D:\Studying\AB Testing\Data source\sample ab_data.csv")

The head() function is used to get the first n rows. 
head() 前5行信息
This function returns the first n rows for the object based on position. It is useful for quickly testing if your object has the right type of data in it.

# Data exploring and data cleaning

In [4]:
df.head()

,user_id,timestamp,group,landing_page,converted
0,851104,11:48.6,control,old_page,0
1,804228,01:45.2,control,old_page,0
2,661590,55:06.2,treatment,new_page,1
3,853541,28:03.1,treatment,new_page,1
4,864975,52:26.2,control,old_page,1


Get an overview of the dataset，check the "null"

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       294478 non-null  int64 
 1   timestamp     294478 non-null  object
 2   group         294478 non-null  object
 3   landing_page  294478 non-null  object
 4   converted     294478 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


In [6]:
((df.group == 'treatment') & (df.landing_page == 'old_page')).sum()

1965

In [7]:
((df.group == 'control')&(df.landing_page == 'new_page')).sum()

1928

In [8]:
df['misaligned']=((df.group == 'treatment')&(df.landing_page == 'old_page'))|((df.group == 'control')&(df.landing_page == 'new_page'))

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       294478 non-null  int64 
 1   timestamp     294478 non-null  object
 2   group         294478 non-null  object
 3   landing_page  294478 non-null  object
 4   converted     294478 non-null  int64 
 5   misaligned    294478 non-null  bool  
dtypes: bool(1), int64(2), object(3)
memory usage: 11.5+ MB


In [10]:
df = df[-df['misaligned']]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 290585 entries, 0 to 294477
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       290585 non-null  int64 
 1   timestamp     290585 non-null  object
 2   group         290585 non-null  object
 3   landing_page  290585 non-null  object
 4   converted     290585 non-null  int64 
 5   misaligned    290585 non-null  bool  
dtypes: bool(1), int64(2), object(3)
memory usage: 13.6+ MB


## Check unique users

In [11]:
df.user_id.nunique()

290584

value_counts()：get a Series containing counts of unique values in descending order
公式Series.value_counts(self, normalize=False, sort=True, ascending=False, bins=None, dropna=True)
sort_values()
公式DataFrame.sort_values(by, *, axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last', ignore_index=False, key=None)

In [12]:
df['user_id'].value_counts().sort_values(ascending = False).head()

773192    2
834487    1
936923    1
679687    1
719014    1
Name: user_id, dtype: int64

In [13]:
df[df['user_id'] == 773192]

,user_id,timestamp,group,landing_page,converted,misaligned
1899,773192,37:58.8,treatment,new_page,1,False
2893,773192,55:59.6,treatment,new_page,0,False


user 773192 重复出现两侧，convert不同(group treatment 选择new_page converted 应该是1，删掉信息记录有误的一条

In [14]:
df.drop(1899, axis = 0, inplace = True)
df['user_id'].value_counts().sort_values(ascending = False).head()

851104    1
834487    1
936923    1
679687    1
719014    1
Name: user_id, dtype: int64

## How many users in each group 

In [15]:
df[['user_id', 'group']].groupby('group').count()

,user_id
group,
control,145274
treatment,145310


In [16]:
df[['user_id','group','converted']].groupby('group').agg({'user_id':'count','converted':'mean'})

,user_id,converted
group,,
control,145274,0.120386
treatment,145310,0.125353


# Data analysis
do hypothesis: new page does not have higher conversion rate than old page at 5% Type I error (alpha)
null: P{new}-P{old}<= 0
alternative: P{new}-P{old} >0

In [17]:
convert_old = df[df.group=='control'].converted.sum()
convert_new = df[df.group=='treatment'].converted.sum()
n_old = len(df[df.group=='control'].converted)
n_new = len(df[df.group=='treatment'].converted)

convert_old, convert_new, n_old, n_new

(17489, 18215, 145274, 145310)

In [19]:
conversion_dic = {'Views':{'Control':n_old,'Test':n_new},'Converts':{'Control':convert_old,'Test':convert_new}}
conversion_table = pd.DataFrame(conversion_dic)
conversion_table['Conversion %'] = conversion_table['Converts']/conversion_table['Views']
conversion_table['Conversion %'] = conversion_table['Conversion %'].apply(lambda x: str(np.round(x,3)*100)+'%')

conversion_table

,Views,Converts,Conversion %
Control,145274,17489,12.0%
Test,145310,18215,12.5%


## Applying Z-test
Way 1: try the z-value function 
Z function: (Avg(X)-Avg(Y))/sqrt of (var(x)/n(x)+var(y)/n(y))
the data is binomial model
mean = p(1-p)
var = nP(1-p)

In [21]:
def z_test(p1,p0,n1,n0):
    delta = p1-p0
    p = (p1*n1 + p0*n0)/(n1+n0)
    return delta / np.sqrt(p*(1-p)*(1/n1+1/n0))


In [22]:
p1 = convert_new /n_new
p0 = convert_old /n_old
n1 = n_new
n0 = n_old


In [23]:
z_value = z_test(p1,p0,n1,n0)
z_value

4.077481782861739

In [24]:
from scipy.stats import norm
p_value = 1-norm.cdf(z_value)

p_value

2.276304781123617e-05

P-value = 2.276304781123617e-05, <0.05. we can reject the null hypothesis and accept the alternative hypothesis that P{new}-P{old} >0
thus, the new page has higher conversion rate than the old page

# Try another way: Z-test in statsmodels
statsmodels.stats.weightstats.ztest(x1,x2=None, value=0)

In [25]:
import statsmodels.api as sm

In [26]:
z_score, p_value = sm.stats.proportions_ztest([convert_new, convert_old], [n_new,n_old], alternative='larger')
z_score, p_value

(4.077481782861739, 2.276304781118429e-05)

the same result with the first way
